Towards 3D Deep Learning for neuropsychiatry: predicting Autism diagnosis using an interpretable Deep Learning pipeline applied to minimally processed structural MRI data, Melanie Garcia, Clare Kelly. medRxiv 2022.10.18.22281196; doi: https://doi.org/10.1101/2022.10.18.22281196

Github: https://github.com/garciaml/Autism-3D-CNN-brain-sMRI?tab=readme-ov-file

# Activate Virtual Environment and Install Requirements
!python3 -m venv ../pretrainedresnet2
!source ../pretrainedresnet2/bin/activate
!python3 -m ipykernel install --user --name=pretrainedresnet2 --display-name "Python (pretrainedresnet2)"
#Switch to notebook/virtual environment kernel

In [8]:
import os
import shutil
import pandas as pd

# Test Paper Reproducibility

In [3]:
def copy_with_structure(source_folder, destination_folder):
    """Copies the contents of source_folder to destination_folder, maintaining the directory structure."""

    for item in os.listdir(source_folder):
        source_path = os.path.join(source_folder, item)
        destination_path = os.path.join(destination_folder, item)

        if os.path.isfile(source_path):
            shutil.copy2(source_path, destination_path)  # copy file with metadata
        elif os.path.isdir(source_path):
             shutil.copytree(source_path, destination_path, dirs_exist_ok=True) # copy directory and its contents
        else:
            print(f"Skipping {source_path}, not a file or directory.")

In [4]:
def count_folders(path):
    folder_count = 0
    for item in os.listdir(path):
        if os.path.isdir(os.path.join(path, item)):
            folder_count += 1
    return folder_count

In [9]:
def count_files(directory_path):
    """Counts the number of files in a directory.

    Args:
        directory_path: The path to the directory.

    Returns:
        The number of files in the directory, or -1 if the directory does not exist.
    """
    if not os.path.exists(directory_path):
        return -1
    
    file_count = 0
    for item in os.listdir(directory_path):
        item_path = os.path.join(directory_path, item)
        if os.path.isfile(item_path):
            file_count += 1
    return file_count

In [11]:
# Model Training - check if can reporduce results from paper (train to 42 epochs with similar split)

#Copy ABIDE1 and ABIDE2 images to ABIDE_COMBINED folder in JustBrain_Data folder
source_folder1 = "JustBrain_Data/ABIDE1"
source_folder2 = "JustBrain_Data/ABIDE2"
destination_folder = "JustBrain_Data/ABIDE_COMBINED"

copy_with_structure(source_folder1, destination_folder)
copy_with_structure(source_folder2, destination_folder)


#Combine ABIDEI and ABIDE2 tsv files
participants1_tsv = pd.read_csv('JustBrain_Data/ABIDE1/participants.tsv', sep="\t", dtype=str)
participants2_tsv = pd.read_csv('JustBrain_Data/ABIDE2/participants.tsv', sep="\t", dtype=str)
participants_ABIDE = pd.concat([participants1_tsv,participants2_tsv])
participants_ABIDE.to_csv('JustBrain_Data/ABIDE_COMBINED/participants.tsv', sep='\t', index=False, header=True)

#Check
print(count_folders(source_folder1) + count_folders(source_folder2))
print(count_folders(destination_folder))
print(participants_ABIDE.shape[0]) # - looks good!


#Copy ABIDE1 and ABIDE2 images to ABIDE_COMBINED folder in Preprocessed_Data folder
source_folder1 = "Preprocessed_Data/ABIDE1"
source_folder2 = "Preprocessed_Data/ABIDE2"
destination_folder = "Preprocessed_Data/ABIDE_COMBINED"

copy_with_structure(source_folder1, destination_folder)
copy_with_structure(source_folder2, destination_folder)

#Check
test_folder = count_files("Preprocessed_Data/ABIDE_COMBINED/test")
train_folder = count_files("Preprocessed_Data/ABIDE_COMBINED/train")
val_folder = count_files("Preprocessed_Data/ABIDE_COMBINED/val")
print(test_folder + train_folder + val_folder)

6
3
2112
2112


In [ ]:
#../Autism-3D-CNN-brain-sMRI/resnet_training/resnet_50.pth'

In [13]:
# ensure you put pre-trained weights in directory
!python ../Autism-3D-CNN-brain-sMRI/train_medicalnet.py 'JustBrain_Data/ABIDE_COMBINED' 'Preprocessed_Data/ABIDE_COMBINED' './outputs/Resnet50/Reproduced_ABIDE_Results' './outputs/Resnet50/Reproduced_ABIDE_Results/model_4_epochs_run1.pth'

/sfs/gpfs/tardis/home/ejh2wy/Autism-3D-CNN-brain-sMRI/resnet2.py:174: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
loading pretrained model ./outputs/Resnet50/Reproduced_ABIDE_Results/model_4_epochs_run1.pth
/sfs/gpfs/tardis/home/ejh2wy/CHMCorr_Autism_Research/../Autism-3D-CNN-brain-sMRI/train_medicalnet.py:103: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode un

# ABIDEII

## Test

In [28]:
participants_tsv = pd.read_csv('JustBrain_Data/ABIDE2/participants.tsv', sep="\t", dtype=str)
participants_tsv.rename(columns={"participant_id" : "SUB_ID"}, inplace=True)
participants_tsv = participants_tsv[participants_tsv.dataset == 'test']
participants_tsv.to_csv('outputs/Resnet50/ABIDE2/test/subjects.csv', index = False)

In [43]:
# Predictions
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py 'Preprocessed_Data/ABIDE2/test' 'outputs/Resnet50/ABIDE2/test/subjects.csv' '../Autism-3D-CNN-brain-sMRI/resnet_training/resnet_50.pth' './outputs/Resnet50/ABIDE2/test'

/sfs/gpfs/tardis/home/ejh2wy/Autism-3D-CNN-brain-sMRI/resnet2.py:174: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
/sfs/gpfs/tardis/home/ejh2wy/CHMCorr_Autism_Research/../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py:104: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_g

## Val

In [38]:
participants_tsv = pd.read_csv('JustBrain_Data/ABIDE2/participants.tsv', sep="\t", dtype=str)
participants_tsv.rename(columns={"participant_id" : "SUB_ID"}, inplace=True)
participants_tsv = participants_tsv[participants_tsv.dataset == 'val']
participants_tsv.to_csv('outputs/Resnet50/ABIDE2/validation/subjects.csv', index = False)

In [39]:
# Predictions
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py 'Preprocessed_Data/ABIDE2/val' 'outputs/Resnet50/ABIDE2/validation/subjects.csv' '../Autism-3D-CNN-brain-sMRI/resnet_training/resnet_50.pth' './outputs/Resnet50/ABIDE2/validation'

/sfs/gpfs/tardis/home/ejh2wy/Autism-3D-CNN-brain-sMRI/resnet2.py:174: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
/sfs/gpfs/tardis/home/ejh2wy/CHMCorr_Autism_Research/../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py:104: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_g

## Train

In [40]:
participants_tsv = pd.read_csv('JustBrain_Data/ABIDE2/participants.tsv', sep="\t", dtype=str)
participants_tsv.rename(columns={"participant_id" : "SUB_ID"}, inplace=True)
participants_tsv = participants_tsv[participants_tsv.dataset == 'train']
participants_tsv.to_csv('outputs/Resnet50/ABIDE2/train/subjects.csv', index = False)

In [41]:
# Predictions
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py 'Preprocessed_Data/ABIDE2/train' 'outputs/Resnet50/ABIDE2/train/subjects.csv' '../Autism-3D-CNN-brain-sMRI/resnet_training/resnet_50.pth' './outputs/Resnet50/ABIDE2/train'

/sfs/gpfs/tardis/home/ejh2wy/Autism-3D-CNN-brain-sMRI/resnet2.py:174: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
/sfs/gpfs/tardis/home/ejh2wy/CHMCorr_Autism_Research/../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py:104: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_g

In [14]:
import pandas as pd

test_predictions_path = 'outputs/Resnet50/ABIDE2/test/test_meta_data.csv'
train_predictions_path = 'outputs/Resnet50/ABIDE2/train/train_meta_data.csv'
val_predictions_path = 'outputs/Resnet50/ABIDE2/validation/validation_meta_data.csv'

truth_path = 'Original_Data/Metadata/ABIDEII_participants.tsv'

test_predictions = pd.read_csv(test_predictions_path)
train_predictions = pd.read_csv(train_predictions_path)
val_predictions = pd.read_csv(val_predictions_path)

truth = pd.read_csv(truth_path, sep="\t", dtype=str)

# Add dataset columns
test_predictions['dataset'] = 'test'
train_predictions['dataset'] = 'train'
val_predictions['dataset'] = 'val'

# Combine
predictions = pd.concat([test_predictions,train_predictions,val_predictions])

# Rename columns to match
predictions.rename(columns={"SUB_ID": "participant_id", "label": "predicted_label"}, inplace=True)
truth.rename(columns={"label": "true_label"}, inplace=True)

# Remove unnecessary columns
predictions = predictions.drop(['filename', 'Unnamed: 0'], axis=1)

# Change to int
predictions['predicted_label'] = predictions['predicted_label'].astype('Int64')
truth['true_label'] = truth['true_label'].astype('Int64')
truth['participant_id'] = truth['participant_id'].astype('Int64')

# Check lengths
print(predictions.shape[0])
print(truth.shape[0])

# Combine
combined = pd.merge(
    left=predictions, 
    right=truth,
    how='left',
    left_on=['participant_id', 'dataset'],
    right_on=['participant_id', 'dataset'],
)

#Reorder
combined = combined[['participant_id', 'dataset', 'true_label', 'predicted_label']]

# Print
combined


1010
1082


,participant_id,dataset,true_label,predicted_label
0,29864,test,1,1
1,29865,test,1,1
2,29866,test,1,1
3,29867,test,1,1
4,29868,test,1,1
...,...,...,...,...
1005,29507,val,1,1
1006,29508,val,1,1
1007,29521,val,1,1
1008,29520,val,0,0


In [15]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(combined['true_label'], combined['predicted_label'])
print(f"Accuracy: {accuracy}")

Accuracy: 1.0


# ACE

In [4]:
# Predictions
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet.py 'JustBrain_Data/ACE' 'Preprocessed_Data/ACE' '../Autism-3D-CNN-brain-sMRI/resnet_training/resnet_50.pth' './outputs/Resnet50/ACE'

/sfs/gpfs/tardis/home/ejh2wy/Autism-3D-CNN-brain-sMRI/resnet2.py:174: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
/sfs/gpfs/tardis/home/ejh2wy/CHMCorr_Autism_Research/../Autism-3D-CNN-brain-sMRI/predict_medicalnet.py:127: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`

# ABIDEI

In [5]:
# Predictions
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet.py 'JustBrain_Data/ABIDE1' 'Preprocessed_Data/ABIDE1' '../Autism-3D-CNN-brain-sMRI/resnet_training/resnet_50.pth' './outputs/Resnet50/ABIDE1'

/sfs/gpfs/tardis/home/ejh2wy/Autism-3D-CNN-brain-sMRI/resnet2.py:174: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
/sfs/gpfs/tardis/home/ejh2wy/CHMCorr_Autism_Research/../Autism-3D-CNN-brain-sMRI/predict_medicalnet.py:127: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`

In [ ]:
# Get Embeddings for xKMEANS